# <div align="center">PIFu: Pixel-Aligned Implicit Function for High-Resolution Clothed Human Digitization</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


***PIFu is a memory efficient and spatiallyaligned 3D representation for 3D surfaces.*** An implicit function defines a surface as a level set of a function f, e.g. f(X) = 0. 

> In mathematics, a ***level set*** of a real-valued function f of n real variables is a set of the form $L_c(f) = \{ (x_1, x_2, ..., x_n) | f(x_1, x_2, ..., x_n) = c \}$, that is, a set where the function takes on a given constant value c. When the number of variables is two, a level set is generically a curve, called a ***level curve***, ***contour line***, or ***isoline***. So a level curve is the set of all real-valued solutions of an equation in two variables x1 and x2. When n = 3, a level set is called a ***level surface*** or ***isosurface***, and for higher values of n the level set is a level ***hypersurface***. So a level surface is the set of all real-valued roots of an equation in three variables x1, x2 and x3, and a level hypersurface is the set of all real-valued roots of an equation in n (n > 3) variables.